In [1]:
!pip install transformers
!pip install discord
!pip install nest_asyncio 

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 32.7 MB/s 
     |████████████████████████████████| 895 kB 18.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 786 kB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 32.3 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 47.7 MB/s 


In [2]:
import pandas as pd
import numpy as np
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from torch.utils.data import Dataset

class LoadDataset(Dataset):
    def __init__(self, df, tk):
        self.df = df
        self.tokenizer = tk

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx, :].values
        text = row[0]
        target = row[1]

        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, max_length=200, pad_to_max_length=True, add_special_tokens=True)

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, target

In [5]:
import torch
from tqdm import tqdm
from transformers import ElectraModel, ElectraTokenizer
import pandas as pd

path = '/content/drive/MyDrive/Data/hate_word/gender_model.pth'

In [17]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")
model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-small-v2-discriminator')

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [18]:
model.load_state_dict(torch.load(path)['model_state_dict'])

<All keys matched successfully>

In [19]:
device = torch.device("cuda:0")
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

In [20]:
def SentenceLoad(sentence):
  print(sentence)
  sentence_df = pd.DataFrame([sentence], columns=['문장'])
  sentence_df['target'] = 1
  test_sentence = LoadDataset(sentence_df, tokenizer)
  return test_sentence

In [21]:
def SentenceCheck(model, test_loader):
  model.eval()
  test_correct = 0
  #test_total = 0
  print('sentence checking')
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
      input_ids_batch = input_ids_batch.to(device)
      attention_masks_batch = attention_masks_batch.to(device)
      y_batch = y_batch.to(device)
      y_batch = y_batch
      y_pred = model(input_ids_batch, attention_mask=attention_masks_batch)[0]
      _, predicted = torch.max(y_pred, 1)
      test_correct += (predicted == y_batch).sum()
      #test_total += len(y_batch)
  print('test result:',test_correct)
  if test_correct == 1:
    return True
  else:
    return False

In [23]:
sentence = '한남이네 ㅋㅋ'
test_sentence = SentenceLoad(sentence)
test_loader = DataLoader(test_sentence, batch_size=32)
SentenceCheck(model, test_loader)

한남이네 ㅋㅋ
sentence checking


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1/1 [00:00<00:00, 25.61it/s]

test result: tensor(1, device='cuda:0')


True

In [ ]:
import discord, asyncio, os
from discord.ext import commands
import nest_asyncio 

nest_asyncio.apply()

game = discord.Game('채팅 검열')
bot = commands.Bot(command_prefix='!', status=discord.Status.online, activity=game)

@bot.command(aliases=['안녕','hi','안녕하세요'])
async def hello(ctx):
    await ctx.send(f'{ctx.author.mention}님 어서오세요')

@bot.command()
async def hihi(ctx):
    await ctx.send('huihui')

@bot.event
async def on_message(message):
    # Bot이 입력한 메시지인 경우
    if message.author.bot:
        return None
    
    test_sentence = SentenceLoad(message.content)
    test_loader = DataLoader(test_sentence, batch_size=32)
    if SentenceCheck(model, test_loader) == True:
      await message.delete()
      await message.channel.send(f'{message.author.mention} 나쁜말 하지마')

bot.run('OTY0MDMxMTE1NjEyNTM2OTAy.Ylet8A.zjD_NOpGsXwaE62u7vm2US3Zjlc')

안녕하세요
sentence checking


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1/1 [00:00<00:00, 40.95it/s]

test result: tensor(0, device='cuda:0')


한남이네 ㅋㅋㅋ
sentence checking


100%|██████████| 1/1 [00:00<00:00, 42.49it/s]

...

100%|██████████| 1/1 [00:00<00:00, 46.13it/s]

test result: tensor(0, device='cuda:0')


여자들은 대학에 가면 안되지
sentence checking


100%|██████████| 1/1 [00:00<00:00, 42.68it/s]

test result: tensor(1, device='cuda:0')
